# Optimización de Portafolio 28 días activo

Este código utiliza la biblioteca **PyPortfolioOpt** para construir un portafolio de inversión que maximiza el **Ratio de Sharpe** ajustado a un horizonte mensual (28 días hábiles). El proceso incluye los siguientes pasos:

In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import yfinance as yf

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

import requests
from bs4 import BeautifulSoup
import pandas as pd

1. **Verifica si el archivo `stocks_adj_close.csv` no existe**:
   - Si **no existe**, lee las acciones desde el archivo `stocks.txt`, agrega el símbolo `^GSPC`, obtiene los precios de cierre ajustados para cada acción, y los guarda en un archivo CSV.
   - Si **ya existe**, simplemente carga los datos desde el archivo CSV.

2. **Elimina la columna de la acción 'TMO'** de los datos.

3. **Define variables**:
   - `assets`: almacena los nombres de las acciones en las columnas del DataFrame.
   - `portafolio`: define el valor del portafolio como 1,000,000.

In [ ]:
if not os.path.exists("stocks_adj_close.csv"):
    with open('stocks.txt', 'r') as file:
        stock_symbols = file.read().splitlines() 

    stock_symbols.insert(0, '^GSPC')

    adj_close_list = []

    for symbol in stock_symbols:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="max")
        adj_close_list.append(hist['Close'].rename(symbol))

    df = pd.concat(adj_close_list, axis=1)
    df.to_csv("stocks_adj_close.csv", index=True)

else:
    df = pd.read_csv("stocks_adj_close.csv", index_col=0)

df = df.drop(columns=['TMO'])
assets = df.columns
portafolio = 1000000

## Cálculo de Rendimientos y Covarianzas Mensuales:
Se calculan los **rendimientos esperados mensuales** (`mu_monthly`) y la **matriz de covarianza mensual** (`s_monthly`) a partir de los datos históricos.
La tasa libre de riesgo también se ajusta al periodo mensual.

Se utiliza la función `max_sharpe()` para calcular los pesos $w_i$ que maximizan el Ratio de Sharpe ajustado para un horizonte temporal (mensual en este caso).

El rendimiento esperado $\mu$ y la matriz de covarianza $S$ se ajustan al horizonte temporal de 1 mes dividiendo por 12:

$$
\mu_{\text{mensual}} = (1 + \mu_{\text{anual}})^{\frac{1}{12}} - 1
$$
$$
S_{\text{mensual}} = \frac{S_{\text{anual}}}{12}
$$

Con esta información, se optimiza el portafolio para maximizar el rendimiento ajustado por riesgo.

Si tu objetivo es maximizar el rendimiento de tu portafolio en un horizonte de **28 días de actividad**, es más apropiado calcular el **Ratio de Sharpe** utilizando la **varianza ajustada a 28 días** en lugar de la varianza diaria. Aquí está el porqué:

**Enfoque en el horizonte temporal**
   Dado que estamos interesados en un rendimiento de 28 días, calcular el Ratio de Sharpe con una varianza ajustada para ese horizonte temporal dará una **medida de riesgo más precisa** para el periodo completo.
   Usar la varianza diaria sin ajustarla a 28 días podría subestimar o sobreestimar el riesgo del portafolio en ese plazo, ya que la volatilidad se acumula con el tiempo.

**Consistencia con los rendimientos esperados**
Al ajustar la varianza a 28 días, estás alineando el cálculo de riesgo con tu **rendimiento esperado en 28 días**, lo que te permite obtener un Ratio de Sharpe más representativo para el periodo que te interesa.
Si usaras la varianza diaria, estarías comparando un rendimiento a 28 días con el riesgo diario, lo cual no reflejaría de manera precisa la relación riesgo-recompensa en ese horizonte.

**Decisiones informadas**
El **Ratio de Sharpe ajustado a 28 días** será una mejor guía para optimizar tu portafolio, porque evalúa tanto el rendimiento como el riesgo en el mismo marco temporal que te importa (28 días).
Esto te permitirá identificar qué activos tienen la mejor compensación entre riesgo y rendimiento durante esos 28 días.

Para maximizar tu portafolio en un horizonte de 28 días, es preferible calcular el **Ratio de Sharpe** usando la **varianza ajustada a 28 días**. Esto te dará una representación más exacta del riesgo y la recompensa en ese periodo específico.

In [ ]:
mu = expected_returns.mean_historical_return(df)
s = risk_models.sample_cov(df)

mu_monthly = (1 + mu) ** (1/12) - 1
s_monthly = s / 12
risk_free_rate_monthly = (1 + 0.01) ** (1/12) - 1

### Optimización del Ratio de Sharpe:
Utilizando el enfoque de la **frontera eficiente**, el código busca la combinación de activos que maximice el *Ratio de Sharpe*. Esto se logra llamando a `ef.max_sharpe()`, que ajusta los pesos del portafolio para optimizar la relación riesgo-recompensa.

La **frontera eficiente** es un concepto clave en la *teoría moderna de portafolios*, desarrollada por Harry Markowitz. Representa la combinación óptima de activos que maximiza el rendimiento esperado para un nivel dado de riesgo, o, alternativamente, minimiza el riesgo para un rendimiento esperado dado. 

### 1. *Rendimiento Esperado*: 

El *rendimiento esperado* \(E[R_p]\) de un portafolio es una combinación ponderada de los rendimientos esperados de cada activo:

\[
E[R_p] = \sum_{i=1}^n w_i E[R_i]
\]

Donde:
- \(E[R_p]\) es el rendimiento esperado del portafolio.
- \(w_i\) es el peso del activo \(i\) en el portafolio (la fracción de capital invertido en el activo \(i\)).
- \(E[R_i]\) es el rendimiento esperado del activo \(i\).
- \(n\) es el número total de activos en el portafolio.

### 2. *Riesgo (Volatilidad)*:

El **riesgo** o **varianza** \(\sigma_p^2\) del portafolio se calcula utilizando la matriz de covarianza de los activos:

\[
\sigma_p^2 = \sum_{i=1}^n \sum_{j=1}^n w_i w_j \text{Cov}(R_i, R_j)
\]

Donde:
- \(\sigma_p^2\) es la varianza del portafolio.
- \(w_i, w_j\) son los pesos de los activos \(i\) y \(j\).
- \(\text{Cov}(R_i, R_j)\) es la covarianza entre los retornos de los activos \(i\) y \(j\).

Si los retornos de los activos no están correlacionados, la varianza se simplifica a una suma de las varianzas individuales ponderadas.

La **frontera eficiente** es la curva que conecta estos portafolios óptimos. Cualquier combinación de activos que esté sobre la curva es "eficiente", porque ofrece el mejor rendimiento posible para un nivel de riesgo determinado.

### **¿Cómo se calcula la frontera eficiente?**

1. *Rendimientos esperados y matriz de covarianza*: Para cada activo en el portafolio, se calcula el *rendimiento esperado* (promedio de los retornos históricos o proyectados) y la *matriz de covarianza* (que mide cómo se mueven los retornos de los activos entre sí). 
   
   - El *rendimiento esperado del portafolio* es una combinación ponderada de los rendimientos esperados de cada activo.
   - El *riesgo del portafolio* es una función de la matriz de covarianza y los pesos de los activos. 

2. *Optimización matemática*: Se usa optimización cuadrática para encontrar la combinación de activos que minimiza el riesgo para un rendimiento dado o maximiza el rendimiento para un riesgo dado. La función objetivo es la minimización de la *varianza del portafolio* sujeta a una restricción sobre el *rendimiento esperado*.

   Para encontrar la **combinación óptima** de activos, se plantea el siguiente problema de optimización. Si deseamos minimizar el riesgo para un rendimiento esperado dado \(E[R_p]\), debemos resolver:

\[
\min_{w_1, w_2, \dots, w_n} \quad (\sigma_p^2)
\]

Sujeto a las siguientes restricciones:
1. La suma de los pesos debe ser 1 (es decir, todo el capital debe estar invertido):
\[
\sum_{i=1}^n w_i = 1
\]

2. El rendimiento esperado del portafolio debe ser igual a un valor deseado \(E[R_p]\):
\[
\sum_{i=1}^n w_i E[R_i] = E[R_p]
\]

### 3. *Maximización del Ratio de Sharpe*:

El **Ratio de Sharpe** mide el rendimiento ajustado por riesgo y se define como:

\[
\text{Sharpe Ratio} = \frac{E[R_p] - r_f}{\sigma_p}
\]

Donde:
- \(r_f\) es la tasa libre de riesgo.
- \(E[R_p]\) es el rendimiento esperado del portafolio.
- \(\sigma_p\) es la desviación estándar (raíz cuadrada de la varianza) del portafolio.

Para maximizar el **Ratio de Sharpe**, debemos encontrar los pesos \(w_i\) que maximicen la siguiente ecuación:

\[
\max_{w_1, w_2, \dots, w_n} \quad \frac{\sum_{i=1}^n w_i E[R_i] - r_f}{\sqrt{\sum_{i=1}^n \sum_{j=1}^n w_i w_j \text{Cov}(R_i, R_j)}}
\]

In [ ]:
ef = EfficientFrontier(mu_monthly, s_monthly) 

### Limpieza y Filtrado de Pesos:

- Los pesos asignados a cada activo se limpian con `ef.clean_weights()` para eliminar valores insignificantes. Solo se mantienen los activos con pesos no nulos (`non_zero_weights`).

### ¿Cómo se escogen los pesos?

Los **pesos** representan la proporción de capital invertido en cada activo. Para calcular los pesos óptimos:

1. **Frontera de media-varianza**: Para cada nivel de rendimiento deseado, se calcula la combinación óptima de activos que minimiza la varianza o riesgo (utilizando la matriz de covarianza). Estos pesos son los que minimizan la volatilidad del portafolio mientras maximizan su rendimiento para ese nivel de riesgo.

2. **Maximización del Ratio de Sharpe**: En lugar de solo optimizar la frontera eficiente, puedes optimizar el **Ratio de Sharpe** (que mide el rendimiento ajustado por riesgo). Aquí, los pesos se ajustan para maximizar la diferencia entre el rendimiento del portafolio y la tasa libre de riesgo, dividido entre la desviación estándar del portafolio.

   - En el código proporcionado, la función `ef.max_sharpe()` escoge los pesos que maximizan el Ratio de Sharpe, lo que implica la mejor relación rendimiento-riesgo en el contexto de un horizonte temporal específico (mensual, en este caso).

### Ejemplo de cálculo de la frontera eficiente:

Supón que tienes dos activos con sus rendimientos esperados y una matriz de covarianza. Los pesos de los activos (por ejemplo, 60% en el activo A y 40% en el activo B) se calculan de tal forma que el rendimiento esperado del portafolio sea máximo para el nivel de riesgo mínimo posible, o viceversa. Repetir esto para diferentes combinaciones de pesos genera la curva de la frontera eficiente.

### Resumen:

- La **frontera eficiente** conecta los portafolios con la mejor combinación de riesgo-rendimiento.
- Se calcula utilizando optimización matemática basada en los **rendimientos esperados** y la **matriz de covarianza**.
- Los **pesos** se seleccionan para minimizar la varianza o maximizar el Ratio de Sharpe, dependiendo de tus objetivos de optimización.

La fórmula de los **pesos óptimos** en un portafolio que maximiza el **Ratio de Sharpe** está basada en la **teoría de portafolios de Markowitz** y se puede derivar resolviendo un problema de optimización cuadrática. Aquí está la fórmula en **Markdown**:

---

### **Fórmula de los Pesos Óptimos**

Los **pesos óptimos** $\mathbf{w}^*$ que maximizan el **Ratio de Sharpe** se obtienen resolviendo el siguiente sistema:

$$
\mathbf{w}^* = \frac{\Sigma^{-1} (\mu - r_f \mathbf{1})}{\mathbf{1}^\top \Sigma^{-1} (\mu - r_f \mathbf{1})}
$$

Donde:
- $\mathbf{w}^*$ es el vector de pesos óptimos.
- $\Sigma^{-1}$ es la **inversa** de la matriz de **covarianza** de los rendimientos de los activos.
- $\mu$ es el vector de los **rendimientos esperados** de los activos.
- $r_f$ es la **tasa libre de riesgo**.
- $\mathbf{1}$ es un vector de unos (del mismo tamaño que $\mu$ ).
- $\mathbf{1}^\top \Sigma^{-1} (\mu - r_f \mathbf{1})$ es una constante de normalización para asegurar que la suma de los pesos sea 1.

---

### **Explicación de la Fórmula**

1. **Matriz de Covarianza** $\Sigma$:
   - Representa el riesgo o la varianza/covarianza entre los distintos activos del portafolio.
   - Al tomar su inversa $\Sigma^{-1}$, estamos reduciendo el riesgo en función de cómo los activos interactúan entre sí.

2. **Vector de Rendimientos** $\mu$:
   - El vector $\mu $contiene los rendimientos esperados de cada activo.
   - Se ajusta por la tasa libre de riesgo $r_f$ para encontrar los **excesos de retorno**.

3. **Maximización del Ratio de Sharpe**:
   - La fórmula busca asignar más peso a los activos con **mayor exceso de rendimiento** en relación a su **riesgo** (covarianza).
   - La constante de normalización $\mathbf{1}^\top \Sigma^{-1} (\mu - r_f \mathbf{1})$ asegura que la suma de los pesos sea igual a 1, como es requerido en un portafolio completamente invertido.

---

Esta fórmula proporciona los **pesos óptimos** de los activos que **maximizan el rendimiento ajustado por riesgo** para un portafolio, con base en los **rendimientos esperados** y la **matriz de covarianza** de los activos.

In [ ]:
weights = ef.max_sharpe(risk_free_rate=risk_free_rate_monthly)
weights = ef.clean_weights()
non_zero_weights = {stock: weight for stock, weight in weights.items() if weight != 0}
ef.portfolio_performance(verbose=True, risk_free_rate=risk_free_rate_monthly)
latest_prices = get_latest_prices(df)


## Asignación Discreta de Activos:
Para hacer el portafolio más realista, se convierte la asignación de pesos en números enteros de acciones, usando `DiscreteAllocation()`. Esta función distribuye la inversión total en función de los precios actuales de los activos.


In [ ]:
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portafolio)
allocation, leftover = da.lp_portfolio()


## Obtención de Nombres de Empresas:
Se utiliza una función personalizada que realiza solicitudes a Yahoo Finance para obtener el nombre de cada empresa basada en su símbolo bursátil.


In [ ]:
def get_company_name(symbol):
    url = f'https://finance.yahoo.com/quote/{symbol}'
    try:
        page = requests.get(url)
        page.raise_for_status()
        soup = BeautifulSoup(page.text, 'html.parser')
        title = soup.find('title').text
        name = title.split(' (')[0]
        return name
    except requests.RequestException as e:
        return None
    except Exception as e:
        return None

company_names = [get_company_name(symbol) for symbol in allocation]
discrete_allocation_list = [allocation[symbol] for symbol in allocation]

portfolio_df = pd.DataFrame({
    'Company Name': company_names,
    'Company Ticker': allocation.keys(),
    'Discrete Allocation': discrete_allocation_list
})


## Salida y Exportación a Excel:
Finalmente, se genera un DataFrame que muestra la asignación discreta de cada activo en el portafolio, junto con los nombres y símbolos de las empresas. El DataFrame se exporta a un archivo Excel para facilitar su consulta.


Este enfoque te permite maximizar el rendimiento de tu portafolio en el corto plazo (28 días hábiles), ajustando tanto el riesgo como el rendimiento esperados a este horizonte temporal específico.

In [3]:
print(portfolio_df)
portfolio_df.to_excel("portfolio_allocation.xlsx", index=False)